In [1]:
import pandas as pd
import numpy as np

In [2]:
output_loc = "/repos/smote_msfb/public_datasets/Burczynski/"

In [8]:
X = pd.read_csv("/repos/smote_msfb/public_datasets/Burczynski/burczynski_inputs.csv", header=None)
y = pd.read_csv("/repos/smote_msfb/public_datasets/Burczynski/burczynski_outputs.csv", header=None)

In [13]:
X.head(3)

,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,...,f_22273,f_22274,f_22275,f_22276,f_22277,f_22278,f_22279,f_22280,f_22281,f_22282
0,80.729,25.993,49.537,312.482,3.402,207.081,30.560,4.463,675.591,3.922,...,23.571,85.841,487.088,2522.788,2.385,4.183,7.959,1.358,10.373,1.622
1,62.985,22.672,70.321,277.637,20.033,192.188,33.655,3.562,529.337,12.347,...,37.698,81.631,443.878,2334.929,6.983,2.432,10.341,0.793,7.436,1.722
2,61.671,24.485,64.504,216.865,8.081,160.758,35.272,4.251,1452.882,12.682,...,43.063,46.394,548.959,2783.692,3.088,1.529,5.810,0.989,3.032,1.486


In [12]:
# Step 3: Rename all columns as f_*
X.columns = [f'f_{i}' for i in range(X.shape[1])]

In [14]:
X.dtypes

f_0        float64
f_1        float64
f_2        float64
f_3        float64
f_4        float64
            ...   
f_22278    float64
f_22279    float64
f_22280    float64
f_22281    float64
f_22282    float64
Length: 22283, dtype: object

In [15]:
# Identify continuous and categorical columns
continuous_cols = X.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()

# 1. Binarize continuous variables
X_bin = X.copy()
for col in continuous_cols:
    cutoff = X_bin[col].quantile(0.65)
    X_bin[col] = (X_bin[col] > cutoff).astype(int)

# 2. One-hot encode categorical columns
X_bin = pd.get_dummies(X_bin, columns=categorical_cols, drop_first=False)

X_bin.head(3)

,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,...,f_22273,f_22274,f_22275,f_22276,f_22277,f_22278,f_22279,f_22280,f_22281,f_22282
0,0,0,0,1,0,1,1,0,0,0,...,0,1,0,1,0,1,1,0,1,0
1,0,0,1,0,1,1,1,0,0,0,...,0,1,0,0,1,0,1,0,1,1
2,0,0,1,0,0,0,1,0,1,0,...,0,0,1,1,1,0,0,0,0,0


In [8]:
# Step 3: Rename all columns as f_*
X_bin.columns = [f'f_{i}' for i in range(X_bin.shape[1])]

In [19]:
type(y)

pandas.core.frame.DataFrame

In [17]:
y.value_counts()

1.0    59
2.0    42
3.0    26
dtype: int64

In [21]:
# Create binary response: 1 for 'imU', 0 otherwise
target = (y.iloc[:, 0] == 3.0).astype(int).rename('target')

# Check the distribution to confirm imbalance
print(target.value_counts())

# If you want to append this target to your feature DataFrame (e.g., X_encoded or X_bin):
final_df = pd.concat([X_bin.reset_index(drop=True), target.reset_index(drop=True)], axis=1)

0    101
1     26
Name: target, dtype: int64


In [22]:
final_df.head()

,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,...,f_22274,f_22275,f_22276,f_22277,f_22278,f_22279,f_22280,f_22281,f_22282,target
0,0,0,0,1,0,1,1,0,0,0,...,1,0,1,0,1,1,0,1,0,0
1,0,0,1,0,1,1,1,0,0,0,...,1,0,0,1,0,1,0,1,1,0
2,0,0,1,0,0,0,1,0,1,0,...,0,1,1,1,0,0,0,0,0,0
3,0,0,0,0,0,1,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
4,1,0,1,0,0,1,1,1,1,1,...,0,0,0,0,0,0,1,1,0,0


In [23]:
final_df.to_csv(output_loc + "final_data.csv.zip", 
                     index=False, 
                     compression={'method': 'zip', 'archive_name': 'final_data.csv'})